[dataset NY] https://data.cityofnewyork.us/Public-Safety/NYPD-Arrest-Data-Year-to-Date-/uip8-fykc

Utilizando o link está acima, baixei o dataset que faz referência as prisões em NYC desde 2006 até o ano de 2020.

Neste notebook os dados foram carregados e passaram por uma manipulação , inclusive criando algumas features ajudar o modelo de classificação de assaltos.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import date

- Carregando os dados da série histórica e os dados do ano de 2020.

In [2]:
df1 = pd.read_csv('dados_hist.csv')
df1.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,Lon_Lat
0,32311380,06/18/2007,511.0,"CONTROLLED SUBSTANCE, POSSESSION 7",235.0,DANGEROUS DRUGS,PL 2200300,M,Q,27,1.0,18-24,M,BLACK,NaN,NaN,NaN,NaN,NaN
1,192799737,01/26/2019,177.0,SEXUAL ABUSE,116.0,SEX CRIMES,PL 1306503,F,M,25,0.0,45-64,M,BLACK,1000555.0,230994.0,40.800694,-73.941109,POINT (-73.94110928599997 40.800694331000045)
2,193260691,02/06/2019,NaN,NaN,NaN,NaN,PL 2203400,F,M,14,0.0,25-44,M,UNKNOWN,986685.0,215375.0,40.757839,-73.991212,POINT (-73.99121211099998 40.75783900300007)
3,149117452,01/06/2016,153.0,RAPE 3,104.0,RAPE,PL 1302503,F,K,67,0.0,25-44,M,BLACK,998032.0,175598.0,40.648650,-73.950336,POINT (-73.95033556299995 40.648650085000035)
4,190049060,11/15/2018,157.0,RAPE 1,104.0,RAPE,PL 1303501,F,K,77,0.0,25-44,M,BLACK,1003606.0,185050.0,40.674583,-73.930222,POINT (-73.93022154099998 40.67458330800008)


In [3]:
df2 = pd.read_csv('dataset.csv')
df2.head()

,ARREST_KEY,ARREST_DATE,PD_CD,PD_DESC,KY_CD,OFNS_DESC,LAW_CODE,LAW_CAT_CD,ARREST_BORO,ARREST_PRECINCT,JURISDICTION_CODE,AGE_GROUP,PERP_SEX,PERP_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,New Georeferenced Column
0,222243929,12/25/2020,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211200,F,B,40,0,25-44,M,BLACK,1007453,233952,40.808798,-73.916184,POINT (-73.91618413199996 40.80879780500004)
1,222115778,12/22/2020,397.0,"ROBBERY,OPEN AREA UNCLASSIFIED",105.0,ROBBERY,PL 1601001,F,M,33,0,18-24,M,BLACK HISPANIC,1001456,247485,40.845956,-73.937813,POINT (-73.93781267199995 40.84595574000008)
2,222090516,12/21/2020,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211200,F,Q,106,0,25-44,M,BLACK,1028605,187930,40.682398,-73.840079,POINT (-73.84007936899997 40.68239828900005)
3,221653577,12/10/2020,105.0,STRANGULATION 1ST,106.0,FELONY ASSAULT,PL 1211200,F,Q,103,0,18-24,M,BLACK,1039602,190480,40.689336,-73.800409,POINT (-73.80040882999998 40.68933550400004)
4,221599602,12/09/2020,729.0,"FORGERY,ETC.,UNCLASSIFIED-FELO",113.0,FORGERY,PL 1702500,F,S,120,0,18-24,M,BLACK,949767,170539,40.634708,-74.124241,POINT (-74.12424111699994 40.634707811000055)


- Excluindo algumas colunas.

In [4]:
df1 = df1.drop(['ARREST_KEY', 'PD_CD', 'PD_DESC', 'KY_CD', 'LAW_CODE','ARREST_PRECINCT', 'JURISDICTION_CODE', 'X_COORD_CD', 'Y_COORD_CD','Lon_Lat'], axis=1)
df2 = df2.drop(['ARREST_KEY', 'PD_CD', 'PD_DESC', 'KY_CD', 'LAW_CODE','ARREST_PRECINCT', 'JURISDICTION_CODE', 'X_COORD_CD', 'Y_COORD_CD','New Georeferenced Column'], axis=1)

- Unindo os 2 datasets.

In [5]:
df = pd.concat([df1,df2])

In [6]:
df.shape

(5153369, 9)

In [7]:
df.isnull().sum()

ARREST_DATE        0
OFNS_DESC       9066
LAW_CAT_CD     18861
ARREST_BORO        8
AGE_GROUP         17
PERP_SEX           0
PERP_RACE          0
Latitude           1
Longitude          1
dtype: int64

In [8]:
df = df.dropna()

In [9]:
df.head()

,ARREST_DATE,OFNS_DESC,LAW_CAT_CD,ARREST_BORO,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude
1,01/26/2019,SEX CRIMES,F,M,45-64,M,BLACK,40.800694,-73.941109
3,01/06/2016,RAPE,F,K,25-44,M,BLACK,40.648650,-73.950336
4,11/15/2018,RAPE,F,K,25-44,M,BLACK,40.674583,-73.930222
5,09/13/2006,CRIMINAL TRESPASS,M,K,45-64,M,BLACK,40.671254,-73.926714
6,10/24/2018,RAPE,F,M,45-64,M,WHITE HISPANIC,40.716196,-73.997491


In [10]:
df.tail()

,ARREST_DATE,OFNS_DESC,LAW_CAT_CD,ARREST_BORO,AGE_GROUP,PERP_SEX,PERP_RACE,Latitude,Longitude
140408,01/07/2020,DANGEROUS WEAPONS,F,K,18-24,M,BLACK,40.688783,-73.947893
140409,01/04/2020,DANGEROUS DRUGS,F,Q,25-44,M,WHITE,40.700591,-73.807743
140410,01/11/2020,FRAUDS,M,S,18-24,M,WHITE HISPANIC,40.638126,-74.087467
140411,01/08/2020,DANGEROUS WEAPONS,F,K,25-44,M,WHITE HISPANIC,40.682289,-73.985233
140412,01/01/2020,INTOXICATED & IMPAIRED DRIVING,M,Q,25-44,F,WHITE HISPANIC,40.746043,-73.934402


In [11]:
#  Renomeando as colunas

df.columns = ['data_da_ocorrencia','descricao_da_ofensa','nivel_da_ofensa','bairro'
                     ,'idade_do_infrator','sexo_do_infrator','raca_do_infrator','latitude','longitude']

In [12]:
df.head()

,data_da_ocorrencia,descricao_da_ofensa,nivel_da_ofensa,bairro,idade_do_infrator,sexo_do_infrator,raca_do_infrator,latitude,longitude
1,01/26/2019,SEX CRIMES,F,M,45-64,M,BLACK,40.800694,-73.941109
3,01/06/2016,RAPE,F,K,25-44,M,BLACK,40.648650,-73.950336
4,11/15/2018,RAPE,F,K,25-44,M,BLACK,40.674583,-73.930222
5,09/13/2006,CRIMINAL TRESPASS,M,K,45-64,M,BLACK,40.671254,-73.926714
6,10/24/2018,RAPE,F,M,45-64,M,WHITE HISPANIC,40.716196,-73.997491


In [13]:
# reorganizando os dados referentes a data 

def date_to_weekday(date):
    weekday_dict = {0:'Monday', 1:'Tuesday', 2: 'Wednesday', 3: 'Thursday', 4: 'Friday', 5: 'Saturday', 6: 'Sunday'}
    date_time_obj = datetime.datetime.strptime(date, '%m/%d/%Y')
    return weekday_dict[date_time_obj.weekday()]

date = df['data_da_ocorrencia'].str.split("/", n = 3, expand = True)
df['ano'] = date[2].astype('int32')
df['dia'] = date[1].astype('int32')
df['mes'] = date[0].astype('int32')
df['dia_da_semana'] = df['data_da_ocorrencia'].apply(date_to_weekday)

# alterando os dados da coluna nivel_da_ofensa

new_nivel_da_ofensa = {'M': "contravencao", 'F': "crime", 'V': "violacao",'I':"outro" }
df['nivel_da_ofensa'] = df['nivel_da_ofensa'].map(new_nivel_da_ofensa)

# alterando os dados da coluna bairro

new_bairro = {'B': 'Bronx', 'S': 'Staten_Island', 'K': 'Brooklyn', 'M': 'Manhattan' , 'Q': 'Queens' }
df['bairro'] = df['bairro'].map(new_bairro)

In [14]:
df["data_da_ocorrencia"] = pd.to_datetime(df["data_da_ocorrencia"]) 

In [15]:
df = df.sort_values('data_da_ocorrencia')

In [16]:
df.descricao_da_ofensa.value_counts().head(30)

DANGEROUS DRUGS                                1088730
ASSAULT 3 & RELATED OFFENSES                    515606
OTHER OFFENSES RELATED TO THEFT                 304190
OTHER STATE LAWS                                241509
PETIT LARCENY                                   221393
FELONY ASSAULT                                  210233
VEHICLE AND TRAFFIC LAWS                        204881
CRIMINAL TRESPASS                               200980
DANGEROUS WEAPONS                               197557
MISCELLANEOUS PENAL LAW                         185927
OTHER TRAFFIC INFRACTION                        175524
ROBBERY                                         161287
OFFENSES AGAINST PUBLIC ADMINISTRATION          156542
POSSESSION OF STOLEN PROPERTY 5                 149875
CRIMINAL MISCHIEF & RELATED OFFENSES            146039
GRAND LARCENY                                   125987
INTOXICATED & IMPAIRED DRIVING                  101637
FORGERY                                          84501
OTHER STAT

## Criando novas features.

In [17]:
df.nivel_da_ofensa.value_counts()

contravencao    3404103
crime           1402221
violacao         293034
outro             26258
Name: nivel_da_ofensa, dtype: int64

In [18]:
df.query('nivel_da_ofensa=="crime"').descricao_da_ofensa.value_counts()

DANGEROUS DRUGS                                286281
FELONY ASSAULT                                 210157
MISCELLANEOUS PENAL LAW                        168508
ROBBERY                                        161287
GRAND LARCENY                                  125987
DANGEROUS WEAPONS                               97119
FORGERY                                         84501
BURGLARY                                        72172
CRIMINAL MISCHIEF & RELATED OFFENSES            54405
POSSESSION OF STOLEN PROPERTY                   18816
GRAND LARCENY OF MOTOR VEHICLE                  16534
OTHER STATE LAWS                                14139
RAPE                                            12482
MURDER & NON-NEGL. MANSLAUGHTER                 12212
INTOXICATED/IMPAIRED DRIVING                    11254
SEX CRIMES                                      11063
VEHICLE AND TRAFFIC LAWS                         9714
THEFT-FRAUD                                      9264
CRIMINAL MISCHIEF & RELATED 

In [19]:
df.query('nivel_da_ofensa=="contravencao"').descricao_da_ofensa.value_counts()

DANGEROUS DRUGS                                802449
ASSAULT 3 & RELATED OFFENSES                   515606
OTHER OFFENSES RELATED TO THEFT                304190
PETIT LARCENY                                  221393
CRIMINAL TRESPASS                              200980
VEHICLE AND TRAFFIC LAWS                       194551
OTHER TRAFFIC INFRACTION                       159415
OFFENSES AGAINST PUBLIC ADMINISTRATION         156522
POSSESSION OF STOLEN PROPERTY 5                149875
INTOXICATED & IMPAIRED DRIVING                 101544
DANGEROUS WEAPONS                              100438
CRIMINAL MISCHIEF & RELATED OFFENSES            91634
OTHER STATE LAWS (NON PENAL LAW)                76902
OFF. AGNST PUB ORD SENSBLTY & RGHTS TO PRIV     66051
PROSTITUTION & RELATED OFFENSES                 45037
OFFENSES INVOLVING FRAUD                        27372
SEX CRIMES                                      26958
OFFENSES AGAINST THE PERSON                     23600
FRAUDS                      

In [20]:
df.query('nivel_da_ofensa=="violacao"').descricao_da_ofensa.value_counts()

OTHER STATE LAWS                          226336
DISORDERLY CONDUCT                         30908
MISCELLANEOUS PENAL LAW                    17419
ADMINISTRATIVE CODE                        13754
HARRASSMENT 2                               2219
LOITERING                                    720
LOITERING/GAMBLING (CARDS, DICE, ETC)        566
NEW YORK CITY HEALTH CODE                    515
PROSTITUTION & RELATED OFFENSES              314
ADMINISTRATIVE CODES                         168
UNLAWFUL POSS. WEAP. ON SCHOOL GROUNDS        63
LOITERING/GAMBLING (CARDS, DIC                26
UNDER THE INFLUENCE, DRUGS                    11
F.C.A. P.I.N.O.S.                             10
NYS LAWS-UNCLASSIFIED VIOLATION                2
LOITERING,BEGGING                              1
UNLAWFUL POSS. WEAP. ON SCHOOL                 1
OTHER STATE LAWS (NON PENAL LAW)               1
Name: descricao_da_ofensa, dtype: int64

## Criando a coluna assalto.

ASSAULT 3 & RELATED OFFENSES/ FELONY ASSAULT:

 - agressão agravada ou agressão envolve circunstâncias que tornam o crime mais grave, como quando a vítima é ameaçada ou experimenta violência significativa que atinge substancialmente mais do que um tapa leve.

ROBBERY/BURGLARY:

 - a ação de tomar bens ilegalmente de uma pessoa ou lugar pela força ou ameaça de força.

GRAND LARCENY/ GRAND LARCENY OF MOTOR VEHICLE:

  - Grande furto/roubo/assalto. 

In [21]:
df['assalto'] = df['descricao_da_ofensa'].apply(lambda x: 1 if x=='FELONY ASSAULT' or x=='ASSAULT 3 & RELATED OFFENSES' or x =='ROBBERY' or x=='BURGLARY' or x=='GRAND LARCENY' or x=='GRAND LARCENY OF MOTOR VEHICLE'  else 0)

In [22]:
df.assalto.value_counts()

0    4023797
1    1101819
Name: assalto, dtype: int64

## Criando a coluna referente a final de semana.

In [23]:
df['fds'] = df['dia_da_semana'].apply(lambda x: 1 if x=='Sunday' or x =='Saturday' else 0)

## Criando a coluna referente a prática de delitos graves.

In [24]:
df['delito'] = df['nivel_da_ofensa'].apply(lambda x: 0 if x=='outro' or x=='violacao' else 1)

## Selecionando apenas os últimos 3 anos e criando um novo dataset.

In [25]:
df[df['data_da_ocorrencia'] >='2018-01-01'].to_csv('dados_arrest_NYC.csv', index=False)